# graph.net._utils.node_lookup_by_address

> TODO fill in description

In [ ]:
#| default_exp graph._utils.node_lookup_by_address

In [ ]:
#| hide
from nbdev.showdoc import *;

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
#|export
from __future__ import annotations
from typing import Type, Tuple
from fastcore.basics import patch_to

import fbdev
from fbdev.comp.port import PortType
from fbdev.graph.net import Address, BaseNode, NodePort, Edge, BaseNodePort

In [ ]:
def find_first_indices(s, a):
    indices = []
    for substr in a:
        index = s.find(substr)
        indices.append(index)
    return indices

# Example usage
s = "This is a test string"
a = ["test", "string", "is", "example"]
result = find_first_indices(s, a)
print(result)  # Output: [10, 15, 5, -1] (-1 indicates the substring is not found)

[10, 15, 2, -1]


In [ ]:
min([('a', 1), ('b', 2), ('c', 3)], key=lambda x: x[1])

('a', 1)

In [ ]:
#|export
__delimiters = {
    BaseNode._address_delimiter : BaseNode,
    Edge._address_delimiter : Edge,
    BaseNodePort._address_delimiter : NodePort,
}
def _get_next_address_part(address:Address) -> Tuple[str, Address, Type]:
    _ds = [(address.find(d), d) for d in __delimiters.keys() if address.find(d) != -1]
    if not _ds:
        return address, '', None
    else:
        d_idx, d = min(_ds, key=lambda x: x[0])
    id, rest_of_address = address[:d_idx], address[d_idx+len(d):]
    return id, rest_of_address, __delimiters[d]

def _get_node_child_by_address(node:Node|Net, address:Address) -> Node|Edge|NodePort:
    id, rest_of_address, child_type = _get_next_address_part(address)
    child_id, rest_of_address_after_child, _ = _get_next_address_part(rest_of_address)
    
    if id == '': id = node.id
    if id != node.id: raise ValueError(f"First part of '{address}' does not match net id '{node.id}'.")
    
    if not child_id: return node
    else:
        if issubclass(child_type, BaseNode):
            if not node.is_net:
                raise ValueError(f"Poorly formatted address '{address}'. Non-net cannot have child nodes.")
            return node.nodes[child_id].get_child_by_address(rest_of_address_after_child)
        elif issubclass(child_type, Edge):
            if not rest_of_address_after_child: raise ValueError(f"Poorly formatted address '{address}'")
            return node.edges[child_id]
        elif issubclass(child_type, BaseNodePort):
            if not rest_of_address_after_child: raise ValueError(f"Poorly formatted address '{address}'")
            port_type_label, port_name = child_id.split('.')
            port_type = PortType.get(port_type_label)
            return node.ports[(port_type, port_name)]
        else:
            raise ValueError(f"Poorly formatted address '{address}'")

In [ ]:
id, rest_of_address, child_type = _get_next_address_part(f'{BaseNode.TOP_NODE_ID}->child|edge0')
assert id == BaseNode.TOP_NODE_ID
assert rest_of_address == 'child|edge0'
assert child_type == BaseNode

In [ ]:
id, rest_of_address, child_type = _get_next_address_part(f'{BaseNode.TOP_NODE_ID}|edge0')
assert id == BaseNode.TOP_NODE_ID
assert rest_of_address == 'edge0'
assert child_type == Edge